**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 4 - Open Science Platform - Cross link User and Project accounts

1 - Load required modules

In [ ]:
from new_helper import *
from super_helper import *
from loguru import logger

2 - Set variables and values

In [ ]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 1

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts, user_account_short_ids  = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts, project_account_short_ids = read_project_accounts_from_jsonld('datasets/projects.json')

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [ ]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [ ]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)



# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
user_account_short_id = user_account_short_ids[json_ld_index]
project_account = project_accounts[json_ld_index]
project_account_short_id = project_account_short_ids[json_ld_index]

logger.debug(f"{user_account['account_id']}")
logger.debug(f"{user_account_short_id['account_id']}")

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-02-09 22:17:49,822 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:17:49,832 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:17:49.833 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 22:17:49.854 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 22:17:49.855 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-09 22:17:49.857 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-09 22:17:49.858 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-09 22:17:49.859 | DEBUG    | integration_helpers

7 - Load the sart contract hash

In [ ]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)

8 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [ ]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id['account_id'], "test")
address = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(address) 

# Process the raw data
account_metadata_cid, project_id = process_raw_data(address) #TO DO - From now

9 - Queries Iroha 1 for the project account.

In [ ]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_account_short_id['account_id'], "test")
details = integration_helpers.get_engine_receipts_result(get_account_hash)



9 - Saves `project_id` in the local file sytem for use in Notebook 4

In [ ]:
# Dumping the variable to JSON
dump_variable(project_id, "Project Id")